# Torrents To Microsft One Drive Downloader

###Install Dependencies
https://www.libtorrent.org/<br>
https://rclone.org/

In [0]:
##!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!wget https://downloads.rclone.org/v1.52.1/rclone-v1.52.1-linux-amd64.deb
!apt install ./rclone-v1.52.1-linux-amd64.deb

### Authenticate One Drive

In [0]:
!rclone config

### Mount One Drive
To stream files we need to mount One Drive.

In [0]:
!mkdir onedrive
!nohup rclone --vfs-cache-mode writes mount onedrive: ./onedrive &

###Just check the drive is mounted

In [0]:
!ls ./onedrive

### Paste the Magnet Link here and press enter to start download
Variable **link** stores the link string.

In [0]:
!mkdir Torrent

link = input("PASTE TORRENT/MAGNET LINK HERE \n") # PASTE TORRENT/MAGNET LINK HERE

import libtorrent as lt
import time
import datetime

ses = lt.session()
ses.listen_on(6881, 6891)
params = {
    'save_path': './Torrent/',
    'storage_mode': lt.storage_mode_t(2)}

print(link)

handle = lt.add_magnet_uri(ses, link, params)
ses.start_dht()

begin = time.time()
print(datetime.datetime.now())

print ('Downloading Metadata...')
while (not handle.has_metadata()):
    time.sleep(1)
print ('Got Metadata, Starting Torrent Download...')

print("Starting", handle.name())

while (handle.status().state != lt.torrent_status.seeding):
    s = handle.status()
    state_str = ['queued', 'checking', 'downloading metadata', \
            'downloading', 'finished', 'seeding', 'allocating']
    print ('%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' % \
            (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000, \
            s.num_peers, state_str[s.state]))
    time.sleep(5)

end = time.time()
print(handle.name(), "COMPLETE")

print("Elapsed Time: ",int((end-begin)//60),"min :", int((end-begin)%60), "sec")

print(datetime.datetime.now())

###Copy the downloaded torrent to One Drive

In [0]:
!cp -r Torrent/* onedrive/